### NLP: Lab 11 (CNNs for NLP)  Using CNNs for NLP
Використання згорткових нейронних мереж для обробки природної мови

In [56]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
#nltk.download("all")
import matplotlib.pyplot as plt
import torch

%matplotlib inline

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [57]:
!pip install fastparquet


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [102]:
import pandas as pd

# Завантаження даних

test = "test-00000-of-00001-fa9b3e8ade89a333.parquet"

parquet_test =  pd.read_parquet(test)

parquet_test.head()


,text,label
0,"Deezer.com 10,406,168 Artist DB\n\nWe have sc...",not_spam
1,🚨 ATTENTION ALL USERS! 🚨\n\n🆘 Are you looking ...,spam
2,I'm working on a stats project to test some of...,not_spam
3,"[[Sorry, I cannot generate inappropriate or sp...",spam
4,L@@k at these Unbelievable diet pills that can...,spam


In [103]:
# Створення масивів texts і labels

# Витягуємо тексти та мітки у numpy масиви
texts = parquet_test['text'].to_numpy()  # якщо стовпець з текстами називається 'text'
labels = parquet_test['label'].to_numpy()  # якщо стовпець з мітками називається 'label'

# Перевірка розмірності
assert len(texts) == len(labels), "Помилка: кількість текстів і міток не співпадає!"
print(f"\nЗавантажено {len(texts)} текстів з мітками.")

# Вивід прикладу даних
print("\nПриклад даних:")
for i in range(3):
    print(f"Текст {i+1}: {texts[i][:50]}... (Мітка: {labels[i]})")


Завантажено 2725 текстів з мітками.

Приклад даних:
Текст 1:  Deezer.com 10,406,168 Artist DB

We have scraped ... (Мітка: not_spam)
Текст 2: 🚨 ATTENTION ALL USERS! 🚨

🆘 Are you looking for a ... (Мітка: spam)
Текст 3: I'm working on a stats project to test some of the... (Мітка: not_spam)


### Sentence pre-processing
Попередня обробка речень

In [104]:
from nltk.tokenize import word_tokenize
from collections import defaultdict
import numpy as np

def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.

    Args:
        texts (List[str]): List of text data

    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        # Tokenize a sentence
        # CODE_START
        tokenized_sent = word_tokenize(sent.lower())  # Токенізація та приведення до нижнього регістру
        # CODE_END

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will be the input to our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        # CODE_START
        input_id = [word2idx.get(token, 1) for token in tokenized_sent]  # Використовуємо 1 (<unk>) для невідомих слів
        # CODE_END
        input_ids.append(input_id)

    return np.array(input_ids)

### Pre-trained vectors

In [105]:
from tqdm import tqdm_notebook

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.

    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initialize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "crawl-300d-2M.vec")
embeddings = torch.tensor(embeddings)

Tokenizing...

Loading pretrained vectors...


/var/folders/d_/m95y8vmd14s17wr96v8ppn_c0000gn/T/ipykernel_22695/725594635.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

There are 10002 / 13847 pretrained vectors found.


### PyTorch data loaders

In [106]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
import numpy as np

# 1. Завантаження даних
test = "test-00000-of-00001-fa9b3e8ade89a333.parquet"
parquet_test = pd.read_parquet(test)

# 2. Підготовка даних
texts = parquet_test['text'].astype(str).tolist()  # Переконуємось, що текст - рядок
labels = parquet_test['label'].values

# 3. Кодування міток
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 4. Токенізація тексту
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')  # Або інша модель
tokenized_data = tokenizer(
    texts,
    padding='max_length',  # Доповнення до максимальної довжини
    truncation=True,       # Обрізання довгих текстів
    max_length=128,       # Максимальна довжина послідовності
    return_tensors='pt'   # Повертаємо PyTorch тензори
)

# 5. Розділення на навчальний та валідаційний набори
train_idx, val_idx = train_test_split(
    np.arange(len(texts)),
    test_size=0.05,        # 5% для валідації
    random_state=42,       # Для відтворюваності
    stratify=encoded_labels # Зберігаємо баланс класів
)

# 6. Функція для створення DataLoader
def create_dataloader(input_ids, attention_masks, labels, batch_size=50, is_train=True):
    dataset = TensorDataset(input_ids, attention_masks, labels)
    sampler = RandomSampler(dataset) if is_train else SequentialSampler(dataset)
    return DataLoader(
        dataset,
        sampler=sampler,
        batch_size=batch_size,
        drop_last=is_train  # Ігноруємо останній неповний батч для навчання
    )

# 7. Створення DataLoader для навчання
train_dataloader = create_dataloader(
    tokenized_data['input_ids'][train_idx],
    tokenized_data['attention_mask'][train_idx],
    torch.tensor(encoded_labels[train_idx]),
    is_train=True
)

# 8. Створення DataLoader для валідації
val_dataloader = create_dataloader(
    tokenized_data['input_ids'][val_idx],
    tokenized_data['attention_mask'][val_idx],
    torch.tensor(encoded_labels[val_idx]),
    is_train=False
)

# Виведення статистики
print("\nДані успішно завантажено!")
print(f"Усього прикладів: {len(texts)}")
print(f"Навчальні дані: {len(train_idx)} прикладів")
print(f"Валідаційні дані: {len(val_idx)} прикладів")
print(f"Кількість батчів для навчання: {len(train_dataloader)} (розмір батча: {train_dataloader.batch_size})")
print(f"Кількість батчів для валідації: {len(val_dataloader)}")


Дані успішно завантажено!
Усього прикладів: 2725
Навчальні дані: 2588 прикладів
Валідаційні дані: 137 прикладів
Кількість батчів для навчання: 51 (розмір батча: 50)
Кількість батчів для валідації: 3


### CNN Model

In [67]:
# Sample configuration:
filter_sizes = [2, 3, 4]
num_filters = [2, 2, 2]

In [94]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    """1D Згорткова Мережа для Класифікації Речень."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        Конструктор класу CNN_NLP.

        Аргументи:
            pretrained_embedding (torch.Tensor): Попередньо навчені ембедінги
                розміру (vocab_size, embed_dim)
            freeze_embedding (bool): Встановіть False для донавчання ембедінгів.
                За замовчуванням: False
            vocab_size (int): Розмір словника, потрібен якщо не використовуються
                попередньо навчені ембедінги.
            embed_dim (int): Розмірність векторів слів. За замовчуванням: 300
            filter_sizes (List[int]): Список розмірів фільтрів. За замовчуванням: [3, 4, 5]
            num_filters (List[int]): Кількість фільтрів для кожного розміру.
                За замовчуванням: [100, 100, 100]
            num_classes (int): Кількість класів. За замовчуванням: 2
            dropout (float): Рівень dropout. За замовчуванням: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Шар ембедінгу
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        # Згорткові шари
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        
        # Повнозв'язний шар
        self.fc = nn.Linear(sum(num_filters), num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        """Прямий прохід через мережу.

        Аргументи:
            input_ids (torch.Tensor): Тензор ідентифікаторів токенів
                розміру (batch_size, max_sent_length)

        Повертає:
            logits (torch.Tensor): Вихідні логіти розміру (batch_size, num_classes)
        """

        # Отримуємо ембедінги з input_ids. Форма виходу: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Змінюємо розмірність для згорткового шару. Форма: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Застосовуємо CNN та ReLU. Форма виходу: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Макс-пулінг. Форма виходу: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # Об'єднуємо результати для подачі у повнозв'язний шар
        # Форма виходу: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)

        # Обчислюємо логіти. Форма виходу: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits


In [95]:
# Встановлюємо пристрій (cpu, cuda або mps)
device = 'cpu'

### Optimizer

In [108]:
import torch.optim as optim

def initialize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Створює екземпляр CNN моделі та оптимізатора."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes та \
    num_filters повинні мати однакову довжину."

    # Створюємо екземпляр CNN моделі
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)

    # Відправляємо модель на пристрій (GPU/CPU)
    cnn_model.to(device)

    # Створюємо оптимізатор RMSprop
    optimizer = optim.RMSprop(cnn_model.parameters(), lr=learning_rate)

    return cnn_model, optimizer

### Training loop 
цикл навчання

In [100]:
import random
import time

# Вказуємо функцію втрат - крос-ентропія
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Встановлює seed для відтворюваності результатів."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Навчання CNN моделі."""

    # Відстежуємо найкращу точність на валідації
    best_accuracy = 0

    # Початок циклу навчання
    print("Початок навчання...\n")
    print(f"{'Епоха':^7} | {'Втрати':^12} | {'Втрати (вал)':^10} | {'Точність (вал)':^9} | {'Час':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Навчання
        # =======================================

        # Відстежуємо час та втрати
        t0_epoch = time.time()
        total_loss = 0

        # Переводимо модель у режим навчання
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Завантажуємо бач на пристрій (GPU/CPU)
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Обнуляємо градієнти з попереднього кроку
            optimizer.zero_grad()

            # Прямий прохід - отримуємо передбачення моделі
            logits = model(b_input_ids)

            # Обчислюємо втрати та акумулюємо їх значення
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Зворотній прохід та оновлення ваг
            loss.backward()
            optimizer.step()

        # Розраховуємо середні втрати за епоху
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Валідація
        # =======================================
        if val_dataloader is not None:
            # Після завершення кожної епохи оцінюємо модель на валідаційному наборі
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Відстежуємо найкращу точність
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            # Виводимо метрики навчання
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")

    print("\n")
    print(f"Навчання завершено! Найкраща точність: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """Оцінка моделі на валідаційному наборі даних після кожної епохи."""
    
    # Переводимо модель у режим оцінки (вимикаємо dropout)
    model.eval()

    # Змінні для відстеження результатів
    val_accuracy = []
    val_loss = []

    # Для кожного бачу валідаційного набору...
    for batch in val_dataloader:
        # Завантажуємо бач на пристрій
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Обчислюємо передбачення (без обчислення градієнтів)
        with torch.no_grad():
            logits = model(b_input_ids)

        # Обчислюємо втрати
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Отримуємо передбачення (класи з найбільшою ймовірністю)
        preds = torch.argmax(logits, dim=1).flatten()

        # Розраховуємо точність (у відсотках)
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Середні значення втрат та точності на валідації
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

### Model evaluation
Оцінка моделі CNN з різними підходами до ембедінгів

In [101]:
# CNN-rand: Вектори слів ініціалізуються випадково
set_seed(42)     # Встановлюємо seed для відтворюваності
cnn_rand, optimizer = initialize_model(vocab_size=len(word2idx), # Розмір словника
                                      embed_dim=300,             # Розмірність векторів
                                      learning_rate=0.25,        # Швидкість навчання
                                      dropout=0.5)               # Рівень dropout

train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=3)

Початок навчання...

 Епоха  |    Втрати    | Втрати (вал) | Точність (вал) |    Час   
------------------------------------------------------------


ValueError: too many values to unpack (expected 2)

In [77]:
# CNN-rand: Вектори слів ініціалізуються випадково
set_seed(42)  # Встановлюємо seed для відтворюваності
cnn_rand, optimizer = initialize_model(
    vocab_size=len(word2idx),  # Розмір словника
    embed_dim=300,             # Розмірність векторів
    learning_rate=0.25,        # Швидкість навчання
    dropout=0.5               # Рівень dropout
)
# Навчання моделі на 3 епохи
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=3)

# CNN-static: Використовуються попередньо навчені вектори fastText (без донавчання)
set_seed(42)
cnn_static, optimizer = initialize_model(
    pretrained_embedding=embeddings,  # Попередньо навчені ембедінги
    freeze_embedding=True,            # Заморожуємо ембедінги (не навчаємо)
    learning_rate=0.25,
    dropout=0.5
)
train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=3)

# CNN-non-static: Попередньо навчені вектори fastText з донавчанням
set_seed(42)
cnn_non_static, optimizer = initialize_model(
    pretrained_embedding=embeddings,
    freeze_embedding=False,  # Донавчаємо ембедінги
    learning_rate=0.25,
    dropout=0.5
)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=3)

NameError: name 'train_dataloader' is not defined

### Test predictions

In [80]:
def predict(text, model=cnn_rand.to("cpu"), max_len=62):
    """Predict probability that a review is positive."""

    # Tokenize, pad and encode text
    tokens = word_tokenize(text.lower())
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]

    # Convert to PyTorch tensors
    input_id = torch.tensor(input_id).unsqueeze(dim=0)

    # Compute logits
    logits = model.forward(input_id)

    #  Compute probability
    probs = F.softmax(logits, dim=1).squeeze(dim=0)

    print(f"This entry is {probs[1] * 100:.2f}% not spam.")